In [1]:
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
from urllib.request import urlopen
import matplotlib.pyplot as plt
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time
from tqdm import tqdm

In [2]:
years = [x for x in range(2014,2022)]
leagues = ['EPL', 'La_liga', 'Bundesliga', 'Serie_A', 'Ligue_1']

In [3]:
BASE_URL = 'https://understat.com/player/{}'


In [4]:
driver = webdriver.Chrome('./chromedriver')

C:\Users\qurra\AppData\Local\Temp\ipykernel_10996\2299280398.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('./chromedriver')


In [6]:
for player_id in tqdm(range(5741,9480)):
    driver.get(BASE_URL.format(player_id))
    time.sleep(0.5)
    table = []
    page_nums = -1

    try:
        while(True):
            html = driver.page_source
            soup = BeautifulSoup(html, 'html.parser')
            player_table = soup.find('div', id='player-history')
            if page_nums == -1:
                player_table_head = player_table.find('thead')
                columns = [x.get_text() for x in player_table_head.find_all('span')]
                columns[3] = 'match_url'
                columns[0] = 'No.'
                pages = soup.find_all('li', class_='page')
                if len(pages) == 0:
                    with open("exceptions.log", "a") as f:
                        f.write("player_id: {}, exception: no records found\n".format(player_id))
                    break
                page_nums = int(pages[-1].string)
            current_page_num = int(soup.find('li', class_='current')['data-page'])
            player_table_body = player_table.find('tbody')
            for index, tr in enumerate(player_table_body.find_all('tr')):
                row = []
                empty_row_flag = 0
                for i, td in enumerate(tr.find_all('td')):
                    if i == 0 and td.get_text() == '':
                        empty_row_flag = 1
                        break
                    if i == 2 or i == 4:
                        row.append(td.string)
                    elif i == 3:
                        row.append(td.find('a')['href'])
                    else:
                        row.append(td.get_text())
                if empty_row_flag == 0:
                    table.append(row)
            if current_page_num == page_nums:
                break
            css_selector = "li[data-page='{}']".format(current_page_num+1)
            next_page_btn = driver.find_element(by=By.CSS_SELECTOR, value=css_selector)
            next_page_btn.click()

        if len(table) != 0:
            table_df = pd.DataFrame(table)
            table_df.columns=columns
            table_df.set_index

            table_df.to_csv('players/{}.csv'.format(player_id))
    
    except Exception as exception_message:
        with open("exceptions.log", "a") as f:
            f.write(
                'player_id: {}, exception: {}\n'.format(player_id, str(exception_message)))
    
    time.sleep(1.1)

100%|██████████| 3739/3739 [2:34:42<00:00,  2.48s/it]  
